In [4]:
from time import sleep
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import os
from selenium.webdriver.support.ui import WebDriverWait
import requests
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import TimeoutException
import datetime
from datetime import datetime, timedelta

In [7]:
driver = webdriver.Chrome()
from_location = 'LAX'
to_location = 'HNL'

date = datetime.strptime('2023-07-01', '%Y-%m-%d')
end_date = datetime.strptime('2023-07-02', '%Y-%m-%d')

while date <= end_date:
    formatted_date = date.strftime('%Y-%m-%d')
    url = f'https://www.kayak.com/flights/{from_location}-{to_location}/{formatted_date}?sort=bestflight_a'
    driver.get(url)
    date += timedelta(days=1)


    wait_time = 20  # increase wait time if needed
    wait = WebDriverWait(driver, wait_time)

    while True:
        try:
            show_more_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
            num_buttons = len(show_more_buttons)

            for i in range(num_buttons):
                show_more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
                driver.execute_script("arguments[0].click();", show_more_button)
                try:
                    wait.until(EC.staleness_of(show_more_button))
                except:
                    pass

            flight_rows = driver.find_elements(By.XPATH, '//div[@class="nrc6-inner"]')
            print(flight_rows)

        except TimeoutException:
            break

[<selenium.webdriver.remote.webelement.WebElement (session="89b738d92416a723ef37047c54badd9b", element="931fe159-06e9-4bc5-b4d7-6daed74b6340")>, <selenium.webdriver.remote.webelement.WebElement (session="89b738d92416a723ef37047c54badd9b", element="49eee7b5-8986-4dd6-bd89-5f1a67710b04")>, <selenium.webdriver.remote.webelement.WebElement (session="89b738d92416a723ef37047c54badd9b", element="eaf3d4ce-ac27-47ee-885c-ebb4eb85a68c")>, <selenium.webdriver.remote.webelement.WebElement (session="89b738d92416a723ef37047c54badd9b", element="6649f83a-2c7b-4132-8303-c2053956428d")>, <selenium.webdriver.remote.webelement.WebElement (session="89b738d92416a723ef37047c54badd9b", element="418cbd91-2e05-4793-901a-da7954fe6c37")>, <selenium.webdriver.remote.webelement.WebElement (session="89b738d92416a723ef37047c54badd9b", element="cfd5f58a-4d6b-41b2-b3eb-8b7f304ca78c")>, <selenium.webdriver.remote.webelement.WebElement (session="89b738d92416a723ef37047c54badd9b", element="8ba19136-52f2-4a65-b5b9-203d0e7c

In [94]:
list_prices = []
list_company_names=[]
list_stops=[]
list_duration=[]

In [6]:

for WebElement in flight_rows:
    elementHTML = WebElement.get_attribute('outerHTML')
    elementSoup = BeautifulSoup(elementHTML, 'html.parser')
    # price
    temp_price = elementSoup.find("div", {"class": "nrc6-price-section"})
    price = temp_price.find("div", {"class": "f8F1-price-text"})
    list_prices.append(price.text)
    
    #company names#
    company_names = elementSoup.find("div",{"class":"c_cgF c_cgF-mod-variant-default"}).text
    list_company_names.append(company_names)
    
    # stops
    temp_stops = elementSoup.find("div", {"class": "vmXl vmXl-mod-variant-default"})
    stops = temp_stops.find("span", {"class": "JWEO-stops-text"})
    list_stops.append(stops.text)

    #durations  
    temp_durations = elementSoup.find("div", {"class": "xdW8"})
    durations = temp_durations.find("div", {"class": "vmXl vmXl-mod-variant-default"}).text
    list_duration.append(durations)



print(list_prices)
print(list_company_names)
print(list_stops)
print(list_duration)


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x000000010dbecae8 chromedriver + 4958952
1   chromedriver                        0x000000010dbe47b3 chromedriver + 4925363
2   chromedriver                        0x000000010d7a6051 chromedriver + 475217
3   chromedriver                        0x000000010d7d91f9 chromedriver + 684537
4   chromedriver                        0x000000010d80fb16 chromedriver + 908054
5   chromedriver                        0x000000010d80b929 chromedriver + 891177
6   chromedriver                        0x000000010d80ade7 chromedriver + 888295
7   chromedriver                        0x000000010d770aca chromedriver + 256714
8   chromedriver                        0x000000010dba89ad chromedriver + 4680109
9   chromedriver                        0x000000010dbadc12 chromedriver + 4701202
10  chromedriver                        0x000000010dbb479f chromedriver + 4728735
11  chromedriver                     

In [8]:
df = pd.DataFrame({
    'Price': list_prices,
    'Company Name': list_company_names,
    'Stops': list_stops,
    'Duration': list_duration,
})

df['Destination'] = to_location
df['From'] = from_location
df['Date'] = date

In [9]:
df

,Price,Company Name,Stops,Duration,Destination,From,Date


In [102]:
df.to_csv('flight_LAX_HNL_data.csv', index=False)